In [ ]:
!pip install customized_konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 881 kB 5.2 MB/s 
     |████████████████████████████████| 465 kB 18.3 MB/s 
     |████████████████████████████████| 19.4 MB 5.9 MB/s 


In [ ]:
import numpy as np
import gensim
import pandas as pd
from konlpy.tag import Okt

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

import random

from sklearn.model_selection import train_test_split


from torch.utils.data import DataLoader

In [ ]:
SEED = 42
#난수 생성기
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True 

In [ ]:
df=pd.read_csv("/content/drive/Shareddrives/22-2 캡스톤 프로젝트/데이터/preprocessed_essay.csv",encoding='utf-8-sig')

In [ ]:
#최종 데이터
df=df[['TEXT_KOR','cEXT','cNEU','cCON']]

In [ ]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer(Okt().morphs)

## 토크나이저 관련

In [ ]:
stopwords = ['의', '가', '이', '은', '들', '는', '과', '도', '를', '으로', '에',
             '하다', '을', '이다', '것', '로', '에서', '그', '인', '서', '네요',
             '임', '랑', '게', '요', '에게', '엔']
tokenizer = Okt()
text_token = []
for sentence in df['TEXT_KOR']:
    tmp = []
    tmp = tokenizer.morphs(sentence, stem=True, norm=True)  #토큰화
    tmp = [word for word in tmp if not word in stopwords]  #불용어 제거
    text_token.append(tmp)

In [ ]:
df['text_token']=text_token

In [ ]:
df

,TEXT_KOR,cEXT,cNEU,cCON,text_token
0,음 지금은 낮잠에서 막 일어났어요 좀 이상하긴 한데 텍사스로 이사 온 이후로 일에 ...,n,y,n,"[음, 지금, 낮잠, 막, 일어나다, 좀, 이상하다, 한데, 텍사스, 이사, 온, ..."
1,자 이제 의식의 흐름 에세이를 시작하겠습니다 고등학교 때 가끔 이런 걸 했었어요 그...,n,n,n,"[자, 이제, 의식, 흐름, 에세이, 시작, 고등학교, 때, 가끔, 이렇다, 걸, ..."
2,누를 수 있는 열린 키보드와 버튼 마침내 효과가 있었고 나는 마침표 쉼표 및 모든 ...,n,y,y,"[누르다, 수, 있다, 열리다, 키, 보드, 와, 버튼, 마침내, 효과, 있다, 나..."
3,믿을 수가 없는 일이야 정말 일어나고 있어요 맥박이 미친 듯이 뛰고 있어요 그래서 ...,y,n,y,"[믿다, 수가, 없다, 일이, 야, 정말, 일어나고, 있다, 맥박, 미치다, 듯이,..."
4,자 여기 다시 한번 좋은 오래된 의식 할당에 대해 말씀드리겠습니다 다시 HS 영어 ...,y,n,n,"[자, 여기, 다시, 한번, 좋다, 오래되다, 의식, 할당, 대해, 말씀드리다, 다..."
...,...,...,...,...,...
2462,집에 왔어요 자고 싶었는데 다음 주 중으로 끝내야 할 심리학 숙제가 생각났어요 아마...,n,y,y,"[집, 오다, 자고, 싶다, 다음, 주, 중, 끝내다, 심리학, 숙제, 생각나다, ..."
2463,의식의 흐름 skdj 그것의 철자가 어떻게 되나요 내가 알기나 하면 나는 오늘 아는...,y,y,n,"[의식, 흐름, skdj, 그것, 철자, 어떻다, 되다, 내, 알기, 나, 나, 오..."
2464,오늘은 12월 8일 수요일이고 이번 학기에는 많은 일들이 있었다 한 학기를 힘차게 ...,n,n,n,"[오늘, 12월, 8일, 수요일, 이고, 이번, 학기, 에는, 많다, 일, 있다, ..."
2465,이번 주는 지옥 같았어 아무튼 이제 20분짜리 글쓰기 과제 시간이에요 나는 지금 매...,n,y,n,"[이번, 주다, 지옥, 같다, 아무튼, 이제, 20분, 짜리, 글쓰기, 과제, 시간..."


In [ ]:
df.to_csv("/content/drive/Shareddrives/22-2 캡스톤 프로젝트/데이터/okt_df.csv",encoding='utf-8-sig',index=False)

# 여기서부터 본격 모델링 관련

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/Shareddrives/22-2 캡스톤 프로젝트/데이터/okt_df.csv',encoding='utf-8-sig')

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_token)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 21308
등장 빈도가 2번 이하인 희귀 단어의 수: 10807
단어 집합에서 희귀 단어의 비율: 50.718040172705095
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.2145197606979434


In [ ]:
# 빈도수 3이하인 단어 제거하고 0 추가한 개수 = vocab_size
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 10502


In [ ]:
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(text_token)

X = tokenizer.texts_to_sequences(text_token)
print('리뷰의 최대 길이 :',max(len(l) for l in X))
print('리뷰의 평균 길이 :',sum(map(len, X))/len(X))

리뷰의 최대 길이 : 1852
리뷰의 평균 길이 : 446.90879610863396


In [ ]:
from keras import preprocessing
from keras_preprocessing.sequence import pad_sequences

max_len=1000
X = pad_sequences(X, maxlen = max_len)
X

array([[   0,    0,    0, ...,   83, 1198, 4535],
       [   0,    0,    0, ..., 1198, 4535,   33],
       [   0,    0,    0, ...,   38,   94, 1524],
       ...,
       [   0,    0,    0, ...,   52,  312,   86],
       [   0,    0,    0, ..., 3969,    5,    2],
       [   0,    0,    0, ...,   10,  510,   86]], dtype=int32)

In [ ]:
y = np.array(df.iloc[:,1:4])
y

array([['n', 'y', 'n'],
       ['n', 'n', 'n'],
       ['n', 'y', 'y'],
       ...,
       ['n', 'n', 'n'],
       ['n', 'y', 'n'],
       ['n', 'y', 'n']], dtype=object)

In [ ]:
y=np.where(y=='n',0,1)
y

array([[0, 1, 0],
       [0, 0, 0],
       [0, 1, 1],
       ...,
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [ ]:
y

array([[0, 1, 0],
       [0, 0, 0],
       [0, 1, 1],
       ...,
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
#X_train = np.array(X_train)
#X_test = np.array(X_test)

In [ ]:
train,test = train_test_split(df, test_size = 0.2, random_state = 42)

In [ ]:
#len(X[train.index])==len(X_train)
X_train=X[train.index]
y_train=y[train.index]

In [ ]:
X_test=X[test.index]
y_test=y[test.index]

In [ ]:
label_names = df.columns[1:4] #label 종류
n_label = len(label_names)

y_train_list = {label_names[i]:y_train[:,i] for i in range(n_label)}
y_test_list = {label_names[i]:y_test[:,i] for i in range(n_label)}

y_train_list

{'cEXT': array([1, 1, 0, ..., 0, 0, 1]),
 'cNEU': array([1, 1, 0, ..., 1, 0, 0]),
 'cCON': array([0, 0, 1, ..., 0, 1, 1])}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
vocab_size=len(tokenizer.word_index)+1

In [ ]:
np.zeros((vocab_size,300))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
import numpy as np
import gensim
word2vec_model=gensim.models.KeyedVectors.load_word2vec_format('/content/drive/Shareddrives/22-2 캡스톤 프로젝트/데이터/GoogleNews-vectors-negative300.bin.gz',binary=True)
embedding_matrix=np.zeros((vocab_size,300))

def get_vector(word):
  if word in word2vec_model:
    return word2vec_model[word]
  else:
    return None

for word, i in tokenizer.word_index.items():
  temp=get_vector(word)
  if temp is not None:
    embedding_matrix[i]=temp


In [ ]:
from torch import nn
weight=torch.FloatTensor(word2vec_model.vectors)

In [ ]:
y_train_1 =np.array([i[0] for i in y_train])

In [ ]:
y_train_1

array([1, 1, 0, ..., 0, 0, 1])

In [ ]:
y_test_1 =np.array([i[0] for i in y_test])

In [ ]:
y_train_2 =np.array([i[1] for i in y_train])
y_test_2 =np.array([i[1] for i in y_test])

y_train_3 =np.array([i[2] for i in y_train])
y_test_3 =np.array([i[2] for i in y_test])

## y_train_1, y_test_1을 y_train_2, y_test_2/ y_train_3, y_test_3으로 바꿔서 train

### Baseline LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, LSTM
from keras.layers import Dropout

model=Sequential()
e=Embedding(vocab_size,300,weights=[embedding_matrix],input_length=1000,trainable=True)
model.add(e)
model.add(LSTM(256, return_sequences=True))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.fit(X_train,y_train_1,epochs=5,verbose=2)

Epoch 1/5
62/62 - 7s - loss: 0.7392 - acc: 0.5073 - 7s/epoch - 107ms/step
Epoch 2/5
62/62 - 4s - loss: 0.4593 - acc: 0.8079 - 4s/epoch - 60ms/step
Epoch 3/5
62/62 - 4s - loss: 0.0708 - acc: 0.9807 - 4s/epoch - 60ms/step
Epoch 4/5
62/62 - 4s - loss: 0.0169 - acc: 0.9959 - 4s/epoch - 60ms/step
Epoch 5/5
62/62 - 4s - loss: 0.0150 - acc: 0.9980 - 4s/epoch - 60ms/step


In [ ]:
model.evaluate(X_test, y_test_1, batch_size=60)

9/9 [==============================] - 1s 46ms/step - loss: 2.5154 - acc: 0.5385


[2.515441417694092, 0.5384615659713745]

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1000, 300)         6392700   
                                                                 
 lstm_4 (LSTM)               (None, 1000, 256)         570368    
                                                                 
 flatten_3 (Flatten)         (None, 256000)            0         
                                                                 
 dense_2 (Dense)             (None, 1)                 256001    
                                                                 
Total params: 7,219,069
Trainable params: 826,369
Non-trainable params: 6,392,700
_________________________________________________________________


In [ ]:
model.evaluate(X_test, y_test_1, batch_size=60)

9/9 [==============================] - 17s 2s/step - loss: 0.6903 - acc: 0.5385


[0.6902790665626526, 0.5384615659713745]

In [ ]:
model.save("/content/drive/Shareddrives/22-2 캡스톤 프로젝트/모델링/LSTM1.h5")

### 과적합 방지를 위한 stratified k-fold cv

In [ ]:
from sklearn.model_selection import StratifiedKFold
n_fold = 5
skf = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, LSTM
from keras.layers import Dropout
accuracy = []
for train, test in skf.split(X_train, y_train_1):
  model=Sequential()
  e=Embedding(vocab_size,300,weights=[embedding_matrix],input_length=1000,trainable=True)
  model.add(e)
  model.add(LSTM(256, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  model.fit(X_train[train],y_train_1[train],epochs=5,verbose=2)
  k_accuracy="%.4f" %(model.evaluate(X_train[test], y_train_1[test])[1])
  accuracy.append(k_accuracy)

# 결과 출력
print("\n %.f fold accuracy: " %n_fold, accuracy)


Epoch 1/5
50/50 - 6s - loss: 0.7521 - accuracy: 0.5032 - 6s/epoch - 121ms/step
Epoch 2/5
50/50 - 3s - loss: 0.5411 - accuracy: 0.7465 - 3s/epoch - 61ms/step
Epoch 3/5
50/50 - 3s - loss: 0.4636 - accuracy: 0.9512 - 3s/epoch - 61ms/step
Epoch 4/5
50/50 - 3s - loss: 0.0233 - accuracy: 0.9975 - 3s/epoch - 61ms/step
Epoch 5/5
50/50 - 3s - loss: 0.0091 - accuracy: 0.9975 - 3s/epoch - 61ms/step
13/13 [==============================] - 1s 26ms/step - loss: 1.7975 - accuracy: 0.5595
Epoch 1/5
50/50 - 5s - loss: 0.7568 - accuracy: 0.5006 - 5s/epoch - 100ms/step
Epoch 2/5
50/50 - 3s - loss: 0.5681 - accuracy: 0.7256 - 3s/epoch - 62ms/step
Epoch 3/5
50/50 - 3s - loss: 0.1487 - accuracy: 0.9493 - 3s/epoch - 62ms/step
Epoch 4/5
50/50 - 3s - loss: 0.0314 - accuracy: 0.9918 - 3s/epoch - 62ms/step
Epoch 5/5
50/50 - 3s - loss: 0.0055 - accuracy: 0.9994 - 3s/epoch - 62ms/step
13/13 [==============================] - 1s 26ms/step - loss: 1.6801 - accuracy: 0.4937
Epoch 1/5
50/50 - 5s - loss: 0.7285 - accu